In [ ]:


from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

from datasets import load_dataset
import pandas as pd
import sys
import config


cloud_config={
    'secure_connect_bundle':config.ASTRA_DB_SECURE_BUNDLE_PATH
}
auth_provider = PlainTextAuthProvider(config.ASTRA_DB_CLIENT_ID,config.ASTRA_DB_CLIENT_SECRET)
cluster=Cluster(cloud=cloud_config,auth_provider=auth_provider)
astraSession = cluster.connect()

llm=OpenAI(openai_api_key=config.OPENAI_API_KEY)
myEmbedding=OpenAIEmbeddings(openai_api_key=config.OPENAI_API_KEY)

myCassandraVStore=Cassandra(
    embedding= myEmbedding,
    session = astraSession,
    keyspace=config.ASTRA_DB_KEYSPACE,
    table_name='test_',
)

print("Loading dataset")
myDataset = pd.read_csv("nutrients_csvfile.csv")
headlines = myDataset[:50]

print("\nGenerating embeddings and storing in AstraDB")
myCassandraVStore.add_texts(headlines)

print("Inserted %i headlines.\n" % len(headlines))

vectorIndex= VectorStoreIndexWrapper(vectorstore=myCassandraVStore)



Loading dataset

Generating embeddings and storing in AstraDB
Inserted 50 headlines.



In [ ]:
first_question=True
while True:
    if first_question:
        query_text= input("\nEnter your question or type 'q' to exit : ")
        first_question=False
    else:
        query_text=input("\nWhat's your next question : ")
    if query_text.lower()=="q":
        break
    
    print("Question: \"%s\"" %query_text)
    answer= vectorIndex.query(query_text,llm=llm).strip()
    print("Answer: \%s\"\n" % answer)




Enter your question or type 'q' to exit :  hi


Question: "hi"
Answer: Hi there!"



What's your next question :  how r u


Question: "how r u"
Answer: I'm doing well. How are you?"



What's your next question :  suggest healthy food


...

Question: "suggest food for keto diet"
Answer: Some keto-friendly foods include meat, fish, eggs, cheese, avocado, and non-starchy vegetables like spinach and kale."



What's your next question :  nutrional value


Question: "nutrional value"
Answer: Calories are a measure of the amount of energy a food provides. They are also a measure of the nutritional value of a food, since the higher the calorie count, the more nutrients the food contains."



What's your next question :  nutrional value


Question: "nutrional value"
Answer: Calories measure the amount of energy that a food provides. However, they do not provide information on the other nutrients a food contains, such as vitamins, minerals, or other macronutrients like fat, protein, and carbohydrates."
